In [7]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

In [44]:
data = pd.read_pickle('data/ema/data3.pickle')

In [46]:
data = data.groupby(['rnum', 'created', 'question_id']).agg({"answer_value": "mean"})


In [49]:
data = data.pivot_table(index=['rnum', 'created'], columns='question_id', values='answer_value')
data.to_csv('data.csv')